# Lab 4.6.4: Option C - AI Agent Swarm with Safety

**Module:** 4.6 - Capstone Project (Domain 4: Production AI)
**Time:** 35-45 hours total
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Project Overview

Build a **multi-agent system** where specialized AI agents collaborate to solve complex tasks:

| Component | Description | DGX Spark Advantage |
|-----------|-------------|---------------------|
| **Agent Orchestra** | 4+ specialized agents | Run multiple LLMs concurrently |
| **Communication** | Inter-agent messaging | Low-latency local inference |
| **Task Planning** | Decompose complex tasks | Large context for planning |
| **Tool Integration** | Each agent has tools | Multiple tool-calling models |
| **Human-in-Loop** | Approval checkpoints | Real-time interaction |
| **Safety Guards** | Action-level guardrails 🛡️ | Safety model alongside agents |

---

## 🎯 Learning Objectives

By completing this project, you will:
- [ ] Design a multi-agent architecture
- [ ] Implement agent communication protocols
- [ ] Build task decomposition and planning
- [ ] Create human-in-the-loop approval flows
- [ ] Implement action-level safety guardrails 🛡️
- [ ] Perform red-team safety evaluation

---

## 🌍 Real-World Context

Multi-agent systems are enabling new capabilities:

| Application | Agent Types | Example Companies |
|-------------|-------------|-------------------|
| **Software Dev** | Planner, Coder, Reviewer, Tester | Devin, GPT-Engineer |
| **Research** | Searcher, Analyzer, Writer, Critic | AutoGPT, BabyAGI |
| **Customer Service** | Router, Specialist, Escalation | AI call centers |
| **Data Analysis** | Collector, Cleaner, Analyzer, Reporter | Automated BI |
| **Content Creation** | Researcher, Writer, Editor, Designer | AI content teams |

Companies like Anthropic, OpenAI, and Microsoft are investing heavily in agent orchestration frameworks.

---

## 🧒 ELI5: What is an Agent Swarm?

> **Imagine you're building a treehouse.** You could:
>
> 1. **Do everything yourself** - slow and you might not be good at everything
> 2. **Get a team of specialists:**
>    - 👷 **Architect** - designs the plan
>    - 🪚 **Carpenter** - builds the structure  
>    - 🎨 **Painter** - makes it look nice
>    - ✅ **Inspector** - checks it's safe
>
> An **AI agent swarm** works the same way!
>
> - **Planner Agent** - breaks down the big task
> - **Specialist Agents** - each does what they're best at
> - **Coordinator** - makes sure they work together
> - **Safety Agent** - stops anyone from doing something dangerous 🛡️
>
> The result? Complex tasks that would overwhelm one AI become manageable when split among specialists who talk to each other.

---

## 🏗️ System Architecture

```
┌─────────────────────────────────────────────────────────────────────────┐
│                        AI Agent Swarm System                            │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│  ┌──────────────┐    ┌──────────────┐    ┌──────────────┐              │
│  │    User      │───▶│   Router/    │───▶│   Response   │              │
│  │  Interface   │    │  Orchestrator│    │  Aggregator  │              │
│  └──────────────┘    └───────┬──────┘    └──────────────┘              │
│                              │                                          │
│                      ┌───────┴───────┐                                  │
│                      │    Planner    │                                  │
│                      │    Agent      │                                  │
│                      └───────┬───────┘                                  │
│                              │                                          │
│     ┌────────────────────────┼────────────────────────┐                 │
│     ▼                        ▼                        ▼                 │
│  ┌──────────┐         ┌──────────┐         ┌──────────┐                │
│  │ Agent 1  │◄───────▶│ Agent 2  │◄───────▶│ Agent 3  │                │
│  │(Research)│         │ (Code)   │         │(Review)  │                │
│  └────┬─────┘         └────┬─────┘         └────┬─────┘                │
│       │                    │                    │                       │
│  ┌────┴────┐          ┌────┴────┐          ┌────┴────┐                 │
│  │  Tools  │          │  Tools  │          │  Tools  │                 │
│  └─────────┘          └─────────┘          └─────────┘                 │
│                              │                                          │
│                      ┌───────┴───────┐                                  │
│                      │    Safety     │ 🛡️                               │
│                      │   Guardian    │                                  │
│                      └───────────────┘                                  │
│                                                                          │
│  Human-in-Loop: Approval required for critical actions                  │
│  Memory Budget: ~60-70GB of 128GB (multiple smaller models)             │
└─────────────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Environment Setup
import os
import sys
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
from enum import Enum
from abc import ABC, abstractmethod
import json
import asyncio

import torch

print("🤖 OPTION C: AI AGENT SWARM WITH SAFETY")
print("="*70)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"\nGPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Memory Planning for Agent Swarm

def plan_option_c_memory():
    """Plan memory for multi-agent system."""
    
    # Strategy: Use smaller, specialized models for each agent
    components = [
        {"name": "Orchestrator (Llama 8B INT4)", "memory_gb": 5.0},
        {"name": "Planner Agent (Llama 8B INT4)", "memory_gb": 5.0},
        {"name": "Research Agent (Llama 8B INT4)", "memory_gb": 5.0},
        {"name": "Coding Agent (CodeLlama 13B INT4)", "memory_gb": 8.0},
        {"name": "Review Agent (Llama 8B INT4)", "memory_gb": 5.0},
        {"name": "Safety Guardian (Llama Guard 8B)", "memory_gb": 4.5},
        {"name": "Embeddings (BGE-M3)", "memory_gb": 1.2},
        {"name": "Shared KV Cache", "memory_gb": 10.0},
        {"name": "Message Queue & State", "memory_gb": 2.0},
        {"name": "System Overhead", "memory_gb": 8.0},
    ]
    
    print("\n💾 DGX SPARK MEMORY PLAN - OPTION C")
    print("="*70)
    print("Strategy: Multiple smaller specialized models")
    print("-"*70)
    
    total = 0
    for comp in components:
        print(f"   {comp['name']:<35} {comp['memory_gb']:>6.1f} GB")
        total += comp['memory_gb']
    
    print("-"*70)
    print(f"   {'TOTAL':<35} {total:>6.1f} GB")
    print(f"   {'REMAINING':<35} {128 - total:>6.1f} GB")
    print(f"\n   ✅ Plenty of room for 5+ agents!")
    
    # Alternative with one large shared model
    print("\n📊 Alternative: Shared Large Model")
    print(f"   Single Llama 70B (INT4): ~38 GB")
    print(f"   + Safety Guard: ~4.5 GB")
    print(f"   + Overhead: ~15 GB")
    print(f"   Total: ~57.5 GB - also fits!")
    
    return total

plan_option_c_memory()

---

## Part 2: Agent Framework

In [ ]:
# Core Agent Classes

class AgentRole(Enum):
    """Predefined agent roles."""
    ORCHESTRATOR = "orchestrator"
    PLANNER = "planner"
    RESEARCHER = "researcher"
    CODER = "coder"
    REVIEWER = "reviewer"
    WRITER = "writer"
    CRITIC = "critic"
    SAFETY = "safety"

@dataclass
class Message:
    """Message between agents."""
    id: str
    sender: str
    recipient: str  # Agent name or "broadcast"
    content: str
    message_type: str = "text"  # text, task, result, approval_request
    metadata: Dict[str, Any] = field(default_factory=dict)
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())

@dataclass
class Task:
    """A task to be executed by an agent."""
    id: str
    description: str
    assigned_to: str
    status: str = "pending"  # pending, in_progress, completed, failed, blocked
    priority: int = 1  # 1=high, 2=medium, 3=low
    dependencies: List[str] = field(default_factory=list)
    result: Optional[str] = None
    requires_approval: bool = False

@dataclass
class AgentConfig:
    """Configuration for an agent."""
    name: str
    role: AgentRole
    model_name: str
    system_prompt: str
    tools: List[str] = field(default_factory=list)
    can_delegate: bool = False
    requires_safety_check: bool = True

class BaseAgent(ABC):
    """
    Base class for all agents in the swarm.
    
    Each agent:
    - Has a specific role and capabilities
    - Can send/receive messages
    - Can use tools
    - Reports to the safety guardian
    """
    
    def __init__(self, config: AgentConfig):
        self.config = config
        self.name = config.name
        self.role = config.role
        self.inbox: List[Message] = []
        self.outbox: List[Message] = []
        self.task_history: List[Task] = []
        self._model = None
        self._message_id_counter = 0
    
    @abstractmethod
    def process_message(self, message: Message) -> Optional[Message]:
        """Process an incoming message and optionally respond."""
        pass
    
    @abstractmethod
    def execute_task(self, task: Task) -> Task:
        """Execute an assigned task."""
        pass
    
    def send_message(self, recipient: str, content: str, msg_type: str = "text") -> Message:
        """Send a message to another agent."""
        self._message_id_counter += 1
        msg = Message(
            id=f"{self.name}_msg_{self._message_id_counter}",
            sender=self.name,
            recipient=recipient,
            content=content,
            message_type=msg_type,
        )
        self.outbox.append(msg)
        return msg
    
    def receive_message(self, message: Message):
        """Receive a message into inbox."""
        self.inbox.append(message)
    
    def get_status(self) -> Dict:
        """Get agent status."""
        return {
            "name": self.name,
            "role": self.role.value,
            "inbox_count": len(self.inbox),
            "outbox_count": len(self.outbox),
            "tasks_completed": len([t for t in self.task_history if t.status == "completed"]),
        }

print("✅ Agent framework defined")

In [ ]:
# Concrete Agent Implementations

class PlannerAgent(BaseAgent):
    """
    Plans and decomposes complex tasks into subtasks.
    
    Responsibilities:
    - Break down user requests into atomic tasks
    - Determine dependencies between tasks
    - Assign tasks to appropriate agents
    - Track overall progress
    """
    
    def __init__(self, config: AgentConfig):
        super().__init__(config)
        self.active_plans: Dict[str, List[Task]] = {}
    
    def decompose_task(self, user_request: str) -> List[Task]:
        """
        Decompose a user request into subtasks.
        
        Returns:
            List of tasks with dependencies
        """
        # In production, use LLM to generate this
        # This is a template structure
        
        plan_prompt = f"""Break down this request into specific tasks:
{user_request}

For each task, specify:
1. Description (what needs to be done)
2. Agent type (researcher, coder, reviewer, writer)
3. Dependencies (which tasks must complete first)
4. Whether it requires human approval

Format as JSON array."""
        
        # Simplified example - would use LLM in production
        tasks = [
            Task(
                id="task_1",
                description="Research background information",
                assigned_to="researcher",
                status="pending",
                priority=1,
            ),
            Task(
                id="task_2",
                description="Implement solution based on research",
                assigned_to="coder",
                status="pending",
                priority=1,
                dependencies=["task_1"],
            ),
            Task(
                id="task_3",
                description="Review implementation",
                assigned_to="reviewer",
                status="pending",
                priority=1,
                dependencies=["task_2"],
                requires_approval=True,
            ),
        ]
        
        return tasks
    
    def process_message(self, message: Message) -> Optional[Message]:
        if message.message_type == "task":
            tasks = self.decompose_task(message.content)
            return self.send_message(
                "orchestrator",
                json.dumps([{"id": t.id, "desc": t.description, "agent": t.assigned_to} for t in tasks]),
                "plan"
            )
        return None
    
    def execute_task(self, task: Task) -> Task:
        task.status = "in_progress"
        subtasks = self.decompose_task(task.description)
        task.result = json.dumps([t.__dict__ for t in subtasks])
        task.status = "completed"
        self.task_history.append(task)
        return task

class ResearchAgent(BaseAgent):
    """Research and gather information."""
    
    def __init__(self, config: AgentConfig, search_tools: List[Callable] = None):
        super().__init__(config)
        self.search_tools = search_tools or []
    
    def research(self, topic: str) -> str:
        """Research a topic using available tools."""
        # In production, use RAG, web search, etc.
        return f"Research findings for: {topic}\n[Simulated research results]"
    
    def process_message(self, message: Message) -> Optional[Message]:
        if message.message_type == "task":
            result = self.research(message.content)
            return self.send_message(message.sender, result, "result")
        return None
    
    def execute_task(self, task: Task) -> Task:
        task.status = "in_progress"
        task.result = self.research(task.description)
        task.status = "completed"
        self.task_history.append(task)
        return task

class CoderAgent(BaseAgent):
    """Write and modify code."""
    
    def generate_code(self, specification: str) -> str:
        """Generate code based on specification."""
        # In production, use code-specialized LLM
        return f"# Generated code for: {specification}\ndef solution():\n    pass"
    
    def process_message(self, message: Message) -> Optional[Message]:
        if message.message_type == "task":
            code = self.generate_code(message.content)
            return self.send_message(message.sender, code, "result")
        return None
    
    def execute_task(self, task: Task) -> Task:
        task.status = "in_progress"
        task.result = self.generate_code(task.description)
        task.status = "completed"
        self.task_history.append(task)
        return task

class ReviewerAgent(BaseAgent):
    """Review and critique outputs."""
    
    def review(self, content: str, content_type: str = "code") -> Dict:
        """Review content and provide feedback."""
        return {
            "approved": True,
            "feedback": "Looks good with minor suggestions",
            "issues": [],
            "suggestions": ["Consider adding error handling"],
        }
    
    def process_message(self, message: Message) -> Optional[Message]:
        if message.message_type == "review_request":
            review = self.review(message.content)
            return self.send_message(message.sender, json.dumps(review), "result")
        return None
    
    def execute_task(self, task: Task) -> Task:
        task.status = "in_progress"
        review = self.review(task.description)
        task.result = json.dumps(review)
        task.status = "completed"
        self.task_history.append(task)
        return task

print("✅ Agent implementations defined")

---

## Part 3: Safety Guardian 🛡️

In [ ]:
# Safety Guardian Agent

@dataclass
class SafetyCheckResult:
    """Result of a safety check."""
    is_safe: bool
    risk_level: str  # low, medium, high, critical
    reason: str
    action: str  # allow, warn, block, require_approval
    recommendations: List[str] = field(default_factory=list)

class SafetyGuardian(BaseAgent):
    """
    Safety guardian that monitors all agent actions.
    
    Responsibilities:
    - Review all agent outputs before delivery
    - Block dangerous actions
    - Flag content for human review
    - Maintain safety audit log
    
    Every action in the swarm passes through this guardian.
    """
    
    def __init__(self, config: AgentConfig):
        super().__init__(config)
        self.audit_log: List[Dict] = []
        self.blocked_count = 0
        self.approved_count = 0
        
        # Dangerous action patterns
        self.dangerous_patterns = [
            r"rm\s+-rf",
            r"sudo\s+",
            r"eval\s*\(",
            r"exec\s*\(",
            r"__import__",
            r"os\.system",
            r"subprocess\.call",
            r"DROP\s+TABLE",
            r"DELETE\s+FROM",
            r"api[_-]?key",
            r"password\s*=",
        ]
        
        # High-risk actions that need approval
        self.approval_required = [
            "file_write",
            "file_delete",
            "code_execution",
            "external_api_call",
            "database_write",
        ]
    
    def check_action(self, agent_name: str, action_type: str, content: str) -> SafetyCheckResult:
        """
        Check if an action is safe to execute.
        
        Args:
            agent_name: Which agent wants to do this
            action_type: Type of action
            content: The content/code to check
            
        Returns:
            SafetyCheckResult with decision
        """
        import re
        
        issues = []
        risk_level = "low"
        
        # Check for dangerous patterns
        for pattern in self.dangerous_patterns:
            if re.search(pattern, content, re.IGNORECASE):
                issues.append(f"Dangerous pattern detected: {pattern}")
                risk_level = "critical"
        
        # Check if approval required
        needs_approval = action_type in self.approval_required
        if needs_approval and risk_level == "low":
            risk_level = "medium"
        
        # Determine action
        if risk_level == "critical":
            action = "block"
            is_safe = False
        elif risk_level == "high" or needs_approval:
            action = "require_approval"
            is_safe = True  # Safe if approved
        elif risk_level == "medium":
            action = "warn"
            is_safe = True
        else:
            action = "allow"
            is_safe = True
        
        result = SafetyCheckResult(
            is_safe=is_safe,
            risk_level=risk_level,
            reason="; ".join(issues) if issues else "No issues detected",
            action=action,
            recommendations=["Review code before execution"] if issues else [],
        )
        
        # Log
        self._log_check(agent_name, action_type, result)
        
        if result.is_safe:
            self.approved_count += 1
        else:
            self.blocked_count += 1
        
        return result
    
    def check_output(self, agent_name: str, output: str) -> SafetyCheckResult:
        """
        Check if agent output is safe to return.
        """
        # Check for PII
        pii_patterns = [
            (r"\b\d{3}-\d{2}-\d{4}\b", "SSN"),
            (r"\b\d{4}[- ]?\d{4}[- ]?\d{4}[- ]?\d{4}\b", "Credit Card"),
            (r"\bpassword\s*[:=]\s*\S+", "Password"),
        ]
        
        import re
        pii_found = []
        for pattern, pii_type in pii_patterns:
            if re.search(pattern, output):
                pii_found.append(pii_type)
        
        if pii_found:
            return SafetyCheckResult(
                is_safe=False,
                risk_level="high",
                reason=f"PII detected: {', '.join(pii_found)}",
                action="block",
                recommendations=["Redact sensitive information before returning"],
            )
        
        return SafetyCheckResult(
            is_safe=True,
            risk_level="low",
            reason="Output passed safety check",
            action="allow",
        )
    
    def _log_check(self, agent: str, action: str, result: SafetyCheckResult):
        """Log a safety check."""
        self.audit_log.append({
            "timestamp": datetime.now().isoformat(),
            "agent": agent,
            "action": action,
            "result": result.action,
            "risk_level": result.risk_level,
            "reason": result.reason,
        })
    
    def get_audit_report(self) -> Dict:
        """Generate audit report."""
        return {
            "total_checks": len(self.audit_log),
            "approved": self.approved_count,
            "blocked": self.blocked_count,
            "by_risk_level": {
                level: len([l for l in self.audit_log if l["risk_level"] == level])
                for level in ["low", "medium", "high", "critical"]
            },
            "recent_blocks": [
                l for l in self.audit_log[-10:] if l["result"] == "block"
            ],
        }
    
    def process_message(self, message: Message) -> Optional[Message]:
        if message.message_type == "safety_check":
            result = self.check_action(
                message.sender,
                message.metadata.get("action_type", "unknown"),
                message.content
            )
            return self.send_message(
                message.sender,
                json.dumps({"safe": result.is_safe, "action": result.action}),
                "safety_result"
            )
        return None
    
    def execute_task(self, task: Task) -> Task:
        task.result = json.dumps(self.get_audit_report())
        task.status = "completed"
        return task

# Test Safety Guardian
print("🛡️ SAFETY GUARDIAN TEST")
print("="*70)

guardian_config = AgentConfig(
    name="safety_guardian",
    role=AgentRole.SAFETY,
    model_name="meta-llama/Llama-Guard-3-8B",
    system_prompt="You are a safety guardian.",
)
guardian = SafetyGuardian(guardian_config)

# Test cases
test_cases = [
    ("coder", "code_write", "def hello(): print('Hello!')"),
    ("coder", "code_write", "import os; os.system('rm -rf /')"),
    ("coder", "file_write", "Writing config to disk"),
]

for agent, action, content in test_cases:
    result = guardian.check_action(agent, action, content)
    icon = "✅" if result.is_safe else "🚫"
    print(f"\n{icon} Agent: {agent}, Action: {action}")
    print(f"   Risk: {result.risk_level} | Action: {result.action}")
    print(f"   Reason: {result.reason}")

print(f"\n📊 Audit Summary: {guardian.get_audit_report()}")

---

## Part 4: Agent Orchestrator

In [ ]:
# Agent Swarm Orchestrator

class AgentSwarmOrchestrator:
    """
    Orchestrates communication and task execution among agents.
    
    Responsibilities:
    - Route messages between agents
    - Manage task queue and execution
    - Enforce safety checks on all actions
    - Handle human-in-the-loop approval
    - Aggregate final results
    """
    
    def __init__(self):
        self.agents: Dict[str, BaseAgent] = {}
        self.safety_guardian: Optional[SafetyGuardian] = None
        self.task_queue: List[Task] = []
        self.completed_tasks: List[Task] = []
        self.message_log: List[Message] = []
        self.pending_approvals: List[Task] = []
    
    def register_agent(self, agent: BaseAgent):
        """Register an agent with the swarm."""
        self.agents[agent.name] = agent
        
        if isinstance(agent, SafetyGuardian):
            self.safety_guardian = agent
        
        print(f"✅ Registered agent: {agent.name} ({agent.role.value})")
    
    def route_message(self, message: Message) -> bool:
        """
        Route a message to its recipient.
        
        All messages pass through safety check first.
        """
        self.message_log.append(message)
        
        # Safety check
        if self.safety_guardian and message.sender != self.safety_guardian.name:
            safety_result = self.safety_guardian.check_output(
                message.sender, message.content
            )
            if not safety_result.is_safe:
                print(f"🛡️ Message blocked: {safety_result.reason}")
                return False
        
        # Route
        if message.recipient == "broadcast":
            for agent in self.agents.values():
                if agent.name != message.sender:
                    agent.receive_message(message)
        elif message.recipient in self.agents:
            self.agents[message.recipient].receive_message(message)
        else:
            print(f"⚠️ Unknown recipient: {message.recipient}")
            return False
        
        return True
    
    def submit_task(self, description: str, requires_approval: bool = False) -> str:
        """
        Submit a new task to the swarm.
        
        Returns:
            Task ID
        """
        task = Task(
            id=f"task_{len(self.task_queue) + len(self.completed_tasks) + 1}",
            description=description,
            assigned_to="planner",  # Always start with planner
            requires_approval=requires_approval,
        )
        self.task_queue.append(task)
        print(f"📋 Task submitted: {task.id}")
        return task.id
    
    def execute_task(self, task: Task) -> Task:
        """
        Execute a single task.
        """
        agent = self.agents.get(task.assigned_to)
        if not agent:
            task.status = "failed"
            task.result = f"No agent found: {task.assigned_to}"
            return task
        
        # Safety check before execution
        if self.safety_guardian and agent.config.requires_safety_check:
            safety_result = self.safety_guardian.check_action(
                agent.name, "task_execution", task.description
            )
            
            if safety_result.action == "block":
                task.status = "blocked"
                task.result = f"Blocked by safety: {safety_result.reason}"
                return task
            
            if safety_result.action == "require_approval" or task.requires_approval:
                task.status = "pending_approval"
                self.pending_approvals.append(task)
                print(f"⏳ Task {task.id} requires human approval")
                return task
        
        # Execute
        print(f"🔄 Executing task {task.id} with {agent.name}")
        return agent.execute_task(task)
    
    def approve_task(self, task_id: str) -> bool:
        """Human approves a pending task."""
        for task in self.pending_approvals:
            if task.id == task_id:
                self.pending_approvals.remove(task)
                task.status = "pending"  # Reset for execution
                executed = self.execute_task(task)
                if executed.status == "completed":
                    self.completed_tasks.append(executed)
                print(f"✅ Task {task_id} approved and executed")
                return True
        return False
    
    def reject_task(self, task_id: str, reason: str = "") -> bool:
        """Human rejects a pending task."""
        for task in self.pending_approvals:
            if task.id == task_id:
                self.pending_approvals.remove(task)
                task.status = "rejected"
                task.result = f"Rejected: {reason}"
                print(f"❌ Task {task_id} rejected")
                return True
        return False
    
    def run_cycle(self):
        """Run one execution cycle."""
        # Process pending tasks
        tasks_to_run = [t for t in self.task_queue if t.status == "pending"]
        
        for task in tasks_to_run:
            # Check dependencies
            deps_complete = all(
                any(ct.id == dep and ct.status == "completed" for ct in self.completed_tasks)
                for dep in task.dependencies
            )
            
            if deps_complete:
                executed = self.execute_task(task)
                if executed.status == "completed":
                    self.task_queue.remove(task)
                    self.completed_tasks.append(executed)
        
        # Process agent outboxes
        for agent in self.agents.values():
            while agent.outbox:
                msg = agent.outbox.pop(0)
                self.route_message(msg)
    
    def get_status(self) -> Dict:
        """Get swarm status."""
        return {
            "agents": len(self.agents),
            "tasks_queued": len(self.task_queue),
            "tasks_completed": len(self.completed_tasks),
            "pending_approvals": len(self.pending_approvals),
            "messages_sent": len(self.message_log),
            "agent_status": {name: agent.get_status() for name, agent in self.agents.items()},
        }

print("✅ AgentSwarmOrchestrator defined")

In [ ]:
# Demo: Create and Run Agent Swarm

print("🤖 AGENT SWARM DEMO")
print("="*70)

# Create orchestrator
swarm = AgentSwarmOrchestrator()

# Create agents
agents_config = [
    AgentConfig("planner", AgentRole.PLANNER, "llama-8b", "You are a planner.", can_delegate=True),
    AgentConfig("researcher", AgentRole.RESEARCHER, "llama-8b", "You research topics."),
    AgentConfig("coder", AgentRole.CODER, "codellama-13b", "You write code."),
    AgentConfig("reviewer", AgentRole.REVIEWER, "llama-8b", "You review outputs."),
]

# Register agents
swarm.register_agent(PlannerAgent(agents_config[0]))
swarm.register_agent(ResearchAgent(agents_config[1]))
swarm.register_agent(CoderAgent(agents_config[2]))
swarm.register_agent(ReviewerAgent(agents_config[3]))
swarm.register_agent(guardian)

print(f"\n📊 Swarm Status: {swarm.get_status()}")

# Submit a task
task_id = swarm.submit_task(
    "Research best practices for Python async programming and create an example",
    requires_approval=False
)

# Run cycle
swarm.run_cycle()

print(f"\n📊 After Cycle: {swarm.get_status()}")

---

## 🎉 Checkpoint

You now have the foundation for an agent swarm:

- ✅ Agent framework with roles and messaging
- ✅ Specialized agents (Planner, Researcher, Coder, Reviewer)
- ✅ Safety Guardian with action-level checks 🛡️
- ✅ Orchestrator for coordination
- ✅ Human-in-the-loop approval system
- ✅ Task queue with dependencies

---

## 🚀 Implementation Roadmap

### Week 1: Foundation
- [ ] Design your agent architecture
- [ ] Implement base agent framework
- [ ] Set up message passing system

### Week 2: Agents
- [ ] Implement 4+ specialized agents
- [ ] Add tool integration for each agent
- [ ] Test individual agent capabilities

### Week 3: Safety & Integration
- [ ] Build Safety Guardian
- [ ] Implement human-in-the-loop
- [ ] End-to-end testing

### Week 4: Evaluation
- [ ] Create evaluation benchmark
- [ ] Red team testing
- [ ] Performance optimization

### Week 5-6: Demo & Docs
- [ ] Build interactive demo
- [ ] Complete technical report
- [ ] Record demo video

In [ ]:
# 🧹 Cleanup
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("✅ Cleanup complete!")
print("\n🎯 Next: Check examples/option-c-agent-swarm/ for complete implementation")